In [1]:
import tensorflow as tf
import keras
import numpy as np
from keras import layers
from keras.layers import MaxPooling2D, AveragePooling2D, UpSampling2D
from keras.layers import Conv2D, Conv2DTranspose
from keras.layers import BatchNormalization, Activation, Dropout
from keras.layers import ZeroPadding2D, Lambda
from keras.layers import Concatenate, Add
from keras.models import Model

def BatchNorm():
    return BatchNormalization(momentum=0.95, epsilon=1e-5)

def common_skip(prev, num_filters, kernel_size, 
                stride_tuple, pad_type, atrous_rate, name):
    """
    The common ResNet block shared by the identity block
    and the convolutional block. Both of those blocks share
    this common functionality.
    """

    prev = BatchNorm()(prev)

    prev = Activation('relu')(prev)

    
    """Syntax:
    keras.layers.Conv2D(filters, kernel_size, strides=(1, 1), 
                    padding='valid', data_format=None, dilation_rate=(1, 1),
                    activation=None, use_bias=True)
    """
    x1 = Conv2D(filters=num_filters, kernel_size=kernel_size, 
                strides=stride_tuple, dilation_rate=atrous_rate,
                padding=pad_type, use_bias=False)(prev)
    if name=="halve_feature_map":
        x1 = MaxPooling2D(pool_size=(2,2),padding='same')(x1)

    x1 = BatchNorm()(x1)
    x1 = Activation('relu')(x1)
    #
    dropout rate is 10%
    x1 = Dropout(rate=0.1)(x1)
    x2 = Conv2D(num_filters, kernel_size=kernel_size, 
                  strides=stride_tuple, dilation_rate=atrous_rate,
                  padding=pad_type, use_bias=False)(x1)
    
    # I think this should be followed by BatchNorm and an activation
    return x2

def convolution_branch(name, num_filters, kernel_size, 
                       stride_tuple, pad_type, atrous_rate, prev):

    prev = Conv2D(num_filters, kernel_size=kernel_size, strides=stride_tuple,
                  padding=pad_type, dilation_rate=atrous_rate, use_bias=False)(prev)
    
    if name=='halve_feature_map':
        # halve the size of feature map by using same padding, 2x2 pooling
        prev = MaxPooling2D(pool_size=(2,2),padding='same')(prev)

    prev = BatchNorm()(prev)
    return prev


def empty_branch(prev):
    return prev

def convolutional_resnet_block(prev_layer, num_filters, name, kernel_size,
                               stride_tuple, pad_type, atrous_rate=1):

    
    block_1 = common_skip(prev=prev_layer, num_filters=num_filters, 
                          name=name, kernel_size=kernel_size, 
                          stride_tuple=stride_tuple,
                          pad_type=pad_type,
                          atrous_rate=atrous_rate)

    block_2 = convolution_branch(name, num_filters=num_filters,
                                 kernel_size=kernel_size, 
                                 stride_tuple=stride_tuple,
                                 prev=prev_layer, 
                                 pad_type=pad_type,
                                 atrous_rate=atrous_rate)
    #print('conv_block',block_1.shape,block_2.shape)
    added = Add()([block_1, block_2])
    
    return added
    

def identity_resnet_block(prev_layer, num_filters, name, kernel_size,
                          stride_tuple, pad_type, atrous_rate=1):
    
    block_1 = common_skip(prev=prev_layer, num_filters=num_filters, 
                          name=name, kernel_size=kernel_size, 
                          stride_tuple=stride_tuple,
                          pad_type=pad_type, 
                          atrous_rate=atrous_rate)
    
    block_2 = empty_branch(prev_layer)
    #print(block_1.shape,block_2.shape)
    added = Add()([block_1, block_2])
    
    return added


def ResNet(input_layer):
    #print('inp',input_layer.shape)
    x = Conv2D(16, (3, 3), strides=(1, 1), padding='same',
                  use_bias=False)(input_layer)
    #print('conv',input_layer.shape)
    x = identity_resnet_block(x, num_filters=16, kernel_size=(3,3),
                              stride_tuple=(1,1), name="identity",
                              pad_type='same', atrous_rate=1)
    
    x = convolutional_resnet_block(x, num_filters=32, kernel_size=(3,3),
                                   stride_tuple=(1,1), name="halve_feature_map",
                                   pad_type='same', atrous_rate=1)
    
    x = convolutional_resnet_block(x, num_filters=64, kernel_size=(3,3),
                                   stride_tuple=(1,1), name="halve_feature_map", 
                                   pad_type='same', atrous_rate=1)
    
    x = identity_resnet_block(x, num_filters=64, kernel_size=(3,3),
                              stride_tuple=(1,1), name="identity",
                              pad_type='same', atrous_rate=1)
    
    x = convolutional_resnet_block(x, num_filters=128, kernel_size=(3,3),
                                   stride_tuple=(1,1), name="halve_feature_map", 
                                   pad_type='same', atrous_rate=1)
    
    x = identity_resnet_block(x, num_filters=128, kernel_size=(3,3),
                              stride_tuple=(1,1), name="identity",
                              pad_type='same', atrous_rate=1)
    
    """ dilated/atrous convolutional ResNet block starts here"""
    
    x = convolutional_resnet_block(x, num_filters=256, kernel_size=(3,3),
                                   stride_tuple=(1,1), name="full_feature_map", 
                                   pad_type='same', atrous_rate=2)
    
    x = identity_resnet_block(x, num_filters=256, kernel_size=(3,3),
                              stride_tuple=(1,1), name="identity",
                              pad_type='same', atrous_rate=2)
    
    x = convolutional_resnet_block(x, num_filters=512, kernel_size=(3,3),
                                   stride_tuple=(1,1), name="full_feature_map", 
                                   pad_type='same', atrous_rate=4)
    
    x = identity_resnet_block(x, num_filters=512, kernel_size=(3,3),
                              stride_tuple=(1,1), name="identity",
                              pad_type='same', atrous_rate=4)
    #Syntax:
    #keras.layers.Conv2D(filters, kernel_size, strides=(1, 1), 
    #                padding='valid', data_format=None, dilation_rate=(1, 1),
    #                activation=None, use_bias=True)
    
    x = Conv2D(filters=512,kernel_size=(3,3),
               strides=(1,1), padding='same',dilation_rate=2)(x)
    x = BatchNorm()(x)
    x = Activation('relu')(x)
    
    x = Conv2D(filters=512,kernel_size=(3,3),
               strides=(1,1), padding='same',dilation_rate=2)(x)
    x = BatchNorm()(x)
    x = Activation('relu')(x)
    
    """End of dilated convolution block"""
    
    x = Conv2D(filters=512,kernel_size=(3,3),
               strides=(1,1), padding='same',dilation_rate=1)(x)
    x = BatchNorm()(x)
    x = Activation('relu')(x)
    
    x = Conv2D(filters=512,kernel_size=(3,3),
               strides=(1,1), padding='same',dilation_rate=1)(x)
    x = BatchNorm()(x)
    x = Activation('relu')(x)
    
    #print('Finished building ResNet')
    return x
    
"""Spatial Pyramid Pooling"""

def upsample_bilinear(in_tensor, new_size):
    resized_height, resized_width = new_size
    return tf.image.resize(images=in_tensor,
                           size=[resized_height,resized_width],
                           method='bilinear',
                           align_corners=True)

def spp_block(prev_layer, pool_size_int, feature_map_shape):

    #kernel = [(1,1),(2,2),(4,4),(8,8)]
    #strides = [(1,1),(2,2),(4,4),(8,8)]
    pool_size_tuple = (pool_size_int, pool_size_int)
    pool_layer = AveragePooling2D(pool_size=pool_size_tuple)(prev_layer)
    conv1 = Conv2D(128, (1, 1), strides=(1, 1),
                        use_bias=False)(pool_layer)
    conv1 = BatchNorm()(conv1)
    conv1 = Activation('relu')(conv1)
    

    # upsampling
    upsampled_layer = Lambda(upsample_bilinear, 
                             arguments={'new_size':feature_map_shape})(conv1)
    #upsampled_layer = UpSampling2D(size=(2, 2),
    #                          interpolation='bilinear')

    return upsampled_layer

def build_pyramid_pooling_module(resnet_last):
    """Build the Pyramid Pooling Module."""
    
    # feature map size to be used for interpolation
    feature_map_size = (16,16) # (height, width) not (width, height)
    pool_sizes = [1,2,4,8]

    pool_block1 = spp_block(resnet_last, pool_sizes[0], feature_map_size)
    pool_block2 = spp_block(resnet_last, pool_sizes[1], feature_map_size)
    pool_block4 = spp_block(resnet_last, pool_sizes[2], feature_map_size)
    pool_block8 = spp_block(resnet_last, pool_sizes[3], feature_map_size)

    # concat all these layers. resulted
    # shape=(1,feature_map_size_x,feature_map_size_y,4096)
    concat = Concatenate()([resnet_last,
                            pool_block8,
                            pool_block4,
                            pool_block2,
                            pool_block1])
    #print(concat.shape)
    return concat    
    

"""
Deconvolution layer comes after concatenation of SPP layers
https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/keras/layers/Conv2DTranspose
From the paper:
"Finally, multi-scale features are fused to obtain an image with 
the same size as the input image by the transposed convolution"
"""
def deconvolution_layer(concat_layer, num_classes, output_shape):
    
    deconv_layer = Conv2DTranspose(filters=num_classes,kernel_size=(16,16),
                                   strides=(1,1),padding='same')(concat_layer)
    
    
    #deconv_layer.set_shape((None,128,128,1))
    deconv_layer = BatchNorm()(deconv_layer)
    deconv_layer = Activation('softmax')(deconv_layer)
    
    # output shape needs to be 128,128, so upsample from 16x16
    output_layer = Lambda(upsample_bilinear,
                          arguments={'new_size':output_shape})(deconv_layer)
                          
    return output_layer

/home/viha4393/anaconda36/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/viha4393/anaconda36/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/viha4393/anaconda36/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/viha4393/anaconda36/lib/python3.7/site-packages/tensorflow/python/f

In [2]:
import os


#from train import build_PSPNet_model, train_val_generator, train_model

from sklearn.model_selection import train_test_split
from keras.layers import Input
train_im_list = os.listdir('./data/train_images/')
train,val = train_test_split(train_im_list,test_size=0.20)

#from ImageDataGenerator import ImageGenerator


input_shape = (128,128)
num_channels=3

input_layer = Input(shape=(input_shape[0],input_shape[1],num_channels))
resnet_block = ResNet(input_layer)
spp_block = build_pyramid_pooling_module(resnet_block)
out_layer = deconvolution_layer(spp_block, num_classes=150, output_shape=(128,128))


In [12]:
model = Model(inputs=input_layer,outputs=out_layer)
from keras.optimizers import Adam
lr = 0.0001
loss_function='mean_squared_error'
adam = Adam(learning_rate=lr)

model.compile(optimizer=adam,
              loss=loss_function,
              metrics=['accuracy'])

model.summary()

In [29]:
""" 
Adapted from sources: 
https://github.com/Vladkryvoruchko/PSPNet-Keras-tensorflow/blob/master/utils/preprocessing.py
https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
"""
import random
from skimage.io import imread
from skimage.transform import resize
from keras.utils import Sequence
import os
import numpy as np

class ImageGenerator(Sequence):
    
    def __init__(self, image_list=None, label_list=None, 
                 image_dir='./train_images', anno_dir='./labels',num_classes = 50, 
                 batch_size = 16, resize_shape_tuple=(128,128), num_channels=3,
                 separator='.',shuffle=True):
        
        self.image_list = image_list
        self.label_list = label_list
        self.image_dir = image_dir
        self.anno_dir = anno_dir
        self.num_classes = num_classes
        self.batch_size = batch_size
        self.resize_shape_tuple = resize_shape_tuple
        self.separator = separator
        self.num_channels = num_channels
        self.shuffle = shuffle
        self.on_epoch_end()


    def __data_generator(self, batch_IDs):
        
        if not os.path.exists(self.image_dir):
            raise FileNotFoundError('ERROR! The folder {} does not'
                                    ' exist\n'.format(self.image_dir))
        
        X = np.zeros([self.batch_size, self.resize_shape_tuple[0], 
                      self.resize_shape_tuple[1], self.num_channels])
        y = np.zeros([self.batch_size, self.resize_shape_tuple[0], 
                      self.resize_shape_tuple[1], self.num_classes])
        
        for i, vals in enumerate(batch_IDs):
            img = resize(imread(os.path.join(self.image_dir, batch_IDs[i])), 
                         self.resize_shape_tuple)
            label = resize(imread(os.path.join(self.anno_dir, batch_IDs[i])),
                       self.resize_shape_tuple)
            #print(y.shape,n_classes)
            label = (np.arange(self.num_classes) == label[:,:,None]).astype('float32')
            #print(y.shape,n_classes)
            assert label.shape[2] == self.num_classes,"Error, dimensions do not match"
            
            X[i,] = img
            
            y[i] = label
            
            
            return X, y
                    
    
    def __len__(self):
            
        return int(np.floor(len(self.image_list)/self.batch_size))

    def __getitem__(self, index):
        
        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        
        batch_IDs = [self.image_list[k] for k in indices]
        # Generate data
        print(batch_IDs)
        X, y = self.__data_generator(batch_IDs)

        return X, y
    
    def on_epoch_end(self):
        self.indices = np.arange(len(self.image_list))
        if self.shuffle == True:
            np.random.shuffle(self.indices)
    

In [30]:
from sklearn.model_selection import train_test_split
image_list = os.listdir('./data/train_images/')
label_list = os.listdir('./data/labels/')
X_train,X_val,y_train,y_val = train_test_split(image_list,label_list,
                                               shuffle=False,test_size=0.2)


In [31]:
train_gen = ImageGenerator(image_list=X_train,
                           label_list=y_train,
                           image_dir='./data/train_images/',
                           anno_dir='./data/labels/',
                           num_classes=150)
val_gen = ImageGenerator(image_list=X_val,
                         label_list=y_val,
                         image_dir='./data/train_images/',
                         anno_dir='./data/labels/',
                         num_classes=150)

In [32]:
model.fit_generator(train_gen,
                    validation_data=val_gen,
                    steps_per_epoch=200,
                    epochs=100,verbose=1)

Epoch 1/100
['2010_002480.jpg', '2011_000713.jpg', '2011_000953.jpg', '2010_005401.jpg', '2011_001536.jpg', '2010_002733.jpg', '2011_003256.jpg', '2011_002350.jpg', '2010_002422.jpg', '2011_000228.jpg', '2011_003078.jpg', '2011_003103.jpg', '2010_002512.jpg', '2011_000469.jpg', '2010_002147.jpg', '2011_001790.jpg']
['2010_005046.jpg', '2007_009413.jpg', '2011_000882.jpg', '2009_001422.jpg', '2011_001601.jpg', '2009_000535.jpg', '2008_005196.jpg', '2009_001941.jpg', '2009_002264.jpg', '2008_002464.jpg', '2007_003541.jpg', '2007_000663.jpg', '2009_003481.jpg', '2009_003589.jpg', '2009_002897.jpg', '2008_005300.jpg']
['2008_000464.jpg', '2009_001411.jpg', '2008_005706.jpg', '2010_005160.jpg', '2008_004345.jpg', '2008_003329.jpg', '2009_000354.jpg', '2007_000175.jpg', '2007_003742.jpg', '2010_000810.jpg', '2008_003814.jpg', '2011_001281.jpg', '2011_002812.jpg', '2008_002210.jpg', '2007_005689.jpg', '2008_002043.jpg']


FileNotFoundError: No such file: '/home/viha4393/workspace/data/labels/2010_005046.jpg'

In [18]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 16) 432         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 128, 128, 16) 64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 128, 128, 16) 0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [28]:
model.output_shape[1:]

(128, 128, 150)